In [42]:
import random
import json_lines
from langdetect import detect_langs
import nltk
from nltk.stem import WordNetLemmatizer
from sklearn.model_selection import train_test_split
import gensim
from gensim.models import KeyedVectors
import numpy as np

LIMIT_INPUT_ROWS= 200
MIN_ENGLISH= 0.0
WNL = WordNetLemmatizer()
REMOVEABLE_CHARS= '[^A-Za-z ]'
ALL_TAGS= ['CC', 'CD', 'DT', 'EX', 'FW', 'IN', 'JJ', 'JJR', 'JJS', 'LS', 'MD', 'NN', 'NNS', 'NNP', 'NNPS', 'PDT',
 'POS', 'PRP', 'PRP$', 'RB', 'RBR', 'RBS', 'RP', 'TO', 'UH', 'VB', 'VBD', 'VBG', 'VBN', 'VBP', 'VBZ', 'WDT', 'WP',
 'WP$', 'WRB']
ACCEPTABLE_TAGS= ['CC', 'CD', 'DT', 'EX', 'FW', 'IN', 'JJ', 'JJR', 'JJS', 'LS', 'MD', 'NN', 'NNS', 'NNP', 'NNPS', 'PDT',
 'POS', 'PRP', 'PRP$', 'RB', 'RBR', 'RBS', 'RP', 'TO', 'UH', 'VB', 'VBD', 'VBG', 'VBN', 'VBP', 'VBZ', 'WDT', 'WP',
 'WP$', 'WRB']
MIN_WORD_OCCURRENCES= 5
VEC_SIZE= 5
MAX_GRAMS= 4
AMOUNT_OF_FILTS= 10 # put to 100 probably

number_entries= 0
with open ('reviews.json', 'rb') as f:
    for item in json_lines.reader(f):
        number_entries+= 1

In [2]:
# READ IN INFO
reviews= []; polarity= []; are_early= []
included_indexs= []
j= 0
while j < LIMIT_INPUT_ROWS:
    randint= random.randint(0, number_entries-1)
    if not randint in included_indexs:
        included_indexs.append(randint)
        j+= 1
    
with open('reviews.json', 'rb') as f: # 5,000 entries in reviews.json
    for counter, item in enumerate(json_lines.reader(f)):
        if counter in included_indexs:
            reviews.append(item['text'])
            polarity.append(item['voted_up'])
            are_early.append(item['early_access'])

# END PRODUCTS: reviews, polarity, are_early

In [3]:
# FILTER TEXT BY CHARACTER

import re
new_reviews= []
for i, text in enumerate(reviews):
    text= re.sub(REMOVEABLE_CHARS, '', text)
    if not str.isspace(text) and text != "":
        new_reviews.append(text)
#     else:
#         print("removed ["+text+"]")
# print(reviews)
# print("########################################################################")
# print(new_reviews)
reviews= new_reviews

# END PRODUCTS: reviews

In [4]:
# FILTER TEXT BY LANGUAGE

filtered_reviews= []
filtered_polarity= []
filtered_are_early= []
for i, text in enumerate(reviews):
    try:
        langs = detect_langs(text)
    except:
        print("["+text+"]")
        print("ERROR: langs failed to define itself. Happens occassionally - just rerun")
        pass
    for lang in langs:
        if str(lang)[0:2] == 'en':
            if float(str(lang)[3:]) > MIN_ENGLISH:
                filtered_reviews.append(reviews[i])
                filtered_polarity.append(polarity[i])
                filtered_are_early.append(are_early[i])
reviews= filtered_reviews
polarity= filtered_polarity
are_early= filtered_are_early

#print(reviews)
# END PRODUCTS: reviews, polarity, are_early

In [5]:
# FILTER TEXT BY WORD TYPE

#print(reviews)
filtered_reviews= []
for i, text in enumerate(reviews):
    #print("-------------------------------------")
    new_text= []
    split_text= text.split()
    tags= nltk.pos_tag(split_text)
    #print(tags)
    #print("#############")
    lemd_split_text= []
    for word in split_text:
        lemd_split_text.append(WNL.lemmatize(word))
    tags= nltk.pos_tag(lemd_split_text)
    #print(tags)
    for j, word in enumerate(lemd_split_text):
        if tags[j][1] in ACCEPTABLE_TAGS:
            new_text.append(word)
    filtered_reviews.append(new_text)
reviews= [' '.join([str(elem) for elem in sublist]) for sublist in filtered_reviews]

# END PRODUCTS: reviews, polarity, are_early

In [6]:
# GET REVIEWS AS LIST OF LISTS

no_sentences= 0
reviews_as_ll= [] # ll denoting the list of list data
for text in reviews:
    split_text= text.split()
    reviews_as_ll.append(split_text)
    no_sentences+= 1

# END PRODUCTS: reviews_as_ll, no_sentences

In [7]:
# MAKE REVIEWS LOWER CASER

for i, review in enumerate(reviews_as_ll):
    for j, word in enumerate(review):
        reviews_as_ll[i][j]= word.lower()

# END PRODUCTS: reviews_as_ll

In [8]:
#X_train, X_test, y_train, y_test= train_test_split(reviews_as_ll, polarity, test_size=0.2, random_state=42)

In [9]:
# MODEL RELATED

model= gensim.models.Word2Vec(min_count= MIN_WORD_OCCURRENCES, vector_size=VEC_SIZE)
model.build_vocab(reviews_as_ll)
model.train(reviews_as_ll, total_examples=no_sentences, epochs=model.epochs) # can be a non-repeatable, 1-pass generator

# Store just the words + their trained embeddings.
word_vectors = model.wv
word_vectors.save("word2vec.wordvectors")

# Load back with memory-mapping = read-only, shared across processes.
wv = KeyedVectors.load("word2vec.wordvectors", mmap='r')

#vector = wv['the']  # Get numpy vector of a word

# END PRODUCTS: wv

In [10]:
# print("####### GAME #######")
# vector = wv['amazing']
# print(vector)
# print("####### PLAYER #######")
# vector = wv['good']
# print(vector)

In [14]:
# Make filters!

filters= []
for i in range(AMOUNT_OF_FILTS):
    filter= []
    n_gram= random.randint(2, MAX_GRAMS)
    for j in range(n_gram):
        filter_layer= []
        for k in range(VEC_SIZE):
            filter_layer.append(random.uniform(-1, 1))
        filter.append(filter_layer)
    filter= np.array([np.array(xi) for xi in filter])
    filters.append(filter)

# END PRODUCTS: filters (list of numpy.ndarray)

[[ 0.4324115   0.65407935 -0.63352008  0.66652726  0.90036906]
 [ 0.30690744 -0.49827128 -0.53644933 -0.10326784 -0.72684   ]
 [ 0.75440297 -0.34256032  0.15292534  0.47799939 -0.62128651]]
3


In [ ]:
#print(reviews_as_ll)
for review in reviews_as_ll:
    greatest_activations= []
    for filt in filters:
        greatest_conv_value= 0
        n_gram= round(filt.size / VEC_SIZE)
        #print(str(len(review)))
        convolutions_per_filt= len(review) - n_gram + 1
        print("convolutions_per_filt="+str(convolutions_per_filt))
        for convolution_i in range(convolutions_per_filt):
            vec_matrix= []
            for i in range(n_gram):
                vec_matrix.append(wv[convolution_i + i])
            vec_matrix= np.array([np.array(xi) for xi in vec_matrix])
            print("\nOLD")
            print(vec_matrix)
            print("NEW")
            vec_matrix= vec_matrix.ravel()
            print(vec_matrix)
            #print(filt)
            #conv_result= numpy.convolve(filt, vec_matrix, 'valid')
            #print(vec_matrix)
            #print("new vec_pos is "+str(vec_pos))
                
            
    
            
    #np.convolve([1,2,3],[0,1,0.5], 'same')
    
# END PRODUCTS: greatest_activations

convolutions_per_filt=3

OLD
[[ 0.6251074   1.1895452   1.8151032  -1.9400036  -1.1596038 ]
 [ 0.39086285  1.0890714   1.5714022  -1.8336313  -0.8715405 ]
 [ 0.6415651   0.8813389   1.2235439  -1.5048789  -0.83719987]]
NEW
[ 0.6251074   1.1895452   1.8151032  -1.9400036  -1.1596038   0.39086285
  1.0890714   1.5714022  -1.8336313  -0.8715405   0.6415651   0.8813389
  1.2235439  -1.5048789  -0.83719987]

OLD
[[ 0.39086285  1.0890714   1.5714022  -1.8336313  -0.8715405 ]
 [ 0.6415651   0.8813389   1.2235439  -1.5048789  -0.83719987]
 [ 0.5081112   0.93254477  1.3595248  -1.8273844  -0.968059  ]]
NEW
[ 0.39086285  1.0890714   1.5714022  -1.8336313  -0.8715405   0.6415651
  0.8813389   1.2235439  -1.5048789  -0.83719987  0.5081112   0.93254477
  1.3595248  -1.8273844  -0.968059  ]

OLD
[[ 0.6415651   0.8813389   1.2235439  -1.5048789  -0.83719987]
 [ 0.5081112   0.93254477  1.3595248  -1.8273844  -0.968059  ]
 [ 0.57446074  0.8889014   1.2783682  -1.3916987  -0.52531874]]
NEW
[ 0.6415651  

  0.47117192  0.6715924  -0.84103537 -0.2958358 ]

OLD
[[ 0.02615885  0.47117192  0.6715924  -0.84103537 -0.2958358 ]
 [ 0.16086034  0.3790927   0.35648713 -0.62765795 -0.40747288]]
NEW
[ 0.02615885  0.47117192  0.6715924  -0.84103537 -0.2958358   0.16086034
  0.3790927   0.35648713 -0.62765795 -0.40747288]

OLD
[[ 0.16086034  0.3790927   0.35648713 -0.62765795 -0.40747288]
 [ 0.2582462   0.39477345  0.6724137  -0.9366188  -0.41219005]]
NEW
[ 0.16086034  0.3790927   0.35648713 -0.62765795 -0.40747288  0.2582462
  0.39477345  0.6724137  -0.9366188  -0.41219005]

OLD
[[ 0.2582462   0.39477345  0.6724137  -0.9366188  -0.41219005]
 [ 0.2856359   0.5284919   0.5581593  -0.94451165 -0.26011017]]
NEW
[ 0.2582462   0.39477345  0.6724137  -0.9366188  -0.41219005  0.2856359
  0.5284919   0.5581593  -0.94451165 -0.26011017]

OLD
[[ 0.2856359   0.5284919   0.5581593  -0.94451165 -0.26011017]
 [ 0.28399548  0.5723237   0.6079228  -0.9472894  -0.2675542 ]]
NEW
[ 0.2856359   0.5284919   0.5581593  -0


OLD
[[ 0.11233407  0.39146596  0.74433786 -0.7801698  -0.52918184]
 [ 0.2964044   0.2534419   0.44968835 -0.38838947 -0.313409  ]
 [ 0.11198283  0.27850512  0.697531   -0.79806286 -0.27205262]
 [ 0.0787584   0.464199    0.58052367 -0.55854523 -0.30952027]]
NEW
[ 0.11233407  0.39146596  0.74433786 -0.7801698  -0.52918184  0.2964044
  0.2534419   0.44968835 -0.38838947 -0.313409    0.11198283  0.27850512
  0.697531   -0.79806286 -0.27205262  0.0787584   0.464199    0.58052367
 -0.55854523 -0.30952027]

OLD
[[ 0.2964044   0.2534419   0.44968835 -0.38838947 -0.313409  ]
 [ 0.11198283  0.27850512  0.697531   -0.79806286 -0.27205262]
 [ 0.0787584   0.464199    0.58052367 -0.55854523 -0.30952027]
 [ 0.14682646  0.5475349   0.64280677 -0.7150407  -0.49777597]]
NEW
[ 0.2964044   0.2534419   0.44968835 -0.38838947 -0.313409    0.11198283
  0.27850512  0.697531   -0.79806286 -0.27205262  0.0787584   0.464199
  0.58052367 -0.55854523 -0.30952027  0.14682646  0.5475349   0.64280677
 -0.7150407  -0

OLD
[[ 0.10136814  0.3998975   0.52438915 -0.5373986  -0.22226697]
 [ 0.31392306  0.43157944  0.529462   -0.816566   -0.3481496 ]
 [ 0.3115335   0.45820466  0.5969806  -1.0709057  -0.60910004]
 [ 0.09961947  0.34290704  0.46897697 -0.82742363 -0.39195433]]
NEW
[ 0.10136814  0.3998975   0.52438915 -0.5373986  -0.22226697  0.31392306
  0.43157944  0.529462   -0.816566   -0.3481496   0.3115335   0.45820466
  0.5969806  -1.0709057  -0.60910004  0.09961947  0.34290704  0.46897697
 -0.82742363 -0.39195433]

OLD
[[ 0.31392306  0.43157944  0.529462   -0.816566   -0.3481496 ]
 [ 0.3115335   0.45820466  0.5969806  -1.0709057  -0.60910004]
 [ 0.09961947  0.34290704  0.46897697 -0.82742363 -0.39195433]
 [ 0.27649724  0.28141782  0.6813284  -0.60779464 -0.43419892]]
NEW
[ 0.31392306  0.43157944  0.529462   -0.816566   -0.3481496   0.3115335
  0.45820466  0.5969806  -1.0709057  -0.60910004  0.09961947  0.34290704
  0.46897697 -0.82742363 -0.39195433  0.27649724  0.28141782  0.6813284
 -0.60779464 -0

 [ 0.3165875   0.5482502   0.4439648  -0.5531978  -0.18563044]]
NEW
[ 0.44207782  0.6872972   0.8547619  -0.91545117 -0.25439793  0.12395953
  0.6132129   0.7094984  -1.0884091  -0.38308296  0.3165875   0.5482502
  0.4439648  -0.5531978  -0.18563044]

OLD
[[ 0.12395953  0.6132129   0.7094984  -1.0884091  -0.38308296]
 [ 0.3165875   0.5482502   0.4439648  -0.5531978  -0.18563044]
 [ 0.18374492  0.29492876  0.7868076  -0.72097623 -0.40329486]]
NEW
[ 0.12395953  0.6132129   0.7094984  -1.0884091  -0.38308296  0.3165875
  0.5482502   0.4439648  -0.5531978  -0.18563044  0.18374492  0.29492876
  0.7868076  -0.72097623 -0.40329486]

OLD
[[ 0.3165875   0.5482502   0.4439648  -0.5531978  -0.18563044]
 [ 0.18374492  0.29492876  0.7868076  -0.72097623 -0.40329486]
 [ 0.08715848  0.26856256  0.5501456  -0.6361616  -0.41629693]]
NEW
[ 0.3165875   0.5482502   0.4439648  -0.5531978  -0.18563044  0.18374492
  0.29492876  0.7868076  -0.72097623 -0.40329486  0.08715848  0.26856256
  0.5501456  -0.636161

...


OLD
[[ 0.05349221  0.42251334  0.390278   -0.4905801  -0.19478743]
 [ 0.324249    0.2905156   0.6615559  -0.45129967 -0.46770763]
 [ 0.01273172  0.15672322  0.51286703 -0.621311   -0.4003294 ]
 [ 0.03023266  0.5466161   0.5809605  -0.86027694 -0.4021706 ]]
NEW
[ 0.05349221  0.42251334  0.390278   -0.4905801  -0.19478743  0.324249
  0.2905156   0.6615559  -0.45129967 -0.46770763  0.01273172  0.15672322
  0.51286703 -0.621311   -0.4003294   0.03023266  0.5466161   0.5809605
 -0.86027694 -0.4021706 ]

OLD
[[ 0.324249    0.2905156   0.6615559  -0.45129967 -0.46770763]
 [ 0.01273172  0.15672322  0.51286703 -0.621311   -0.4003294 ]
 [ 0.03023266  0.5466161   0.5809605  -0.86027694 -0.4021706 ]
 [ 0.23484507  0.23459935  0.58051765 -0.6769568  -0.22333413]]
NEW
[ 0.324249    0.2905156   0.6615559  -0.45129967 -0.46770763  0.01273172
  0.15672322  0.51286703 -0.621311   -0.4003294   0.03023266  0.5466161
  0.5809605  -0.86027694 -0.4021706   0.23484507  0.23459935  0.58051765
 -0.6769568  -0.

OLD
[[ 0.33708748  0.40428442  0.5392641  -0.83261144 -0.12218547]
 [ 0.01714617  0.20603657  0.4504122  -0.48793542 -0.18296939]
 [ 0.22635117  0.21402073  0.30195048 -0.43837684 -0.15800509]
 [ 0.06160438  0.44512972  0.5648724  -0.5935517  -0.45970652]]
NEW
[ 0.33708748  0.40428442  0.5392641  -0.83261144 -0.12218547  0.01714617
  0.20603657  0.4504122  -0.48793542 -0.18296939  0.22635117  0.21402073
  0.30195048 -0.43837684 -0.15800509  0.06160438  0.44512972  0.5648724
 -0.5935517  -0.45970652]

OLD
[[ 0.01714617  0.20603657  0.4504122  -0.48793542 -0.18296939]
 [ 0.22635117  0.21402073  0.30195048 -0.43837684 -0.15800509]
 [ 0.06160438  0.44512972  0.5648724  -0.5935517  -0.45970652]
 [ 0.25720185  0.42276376  0.68241495 -0.6886472  -0.29869643]]
NEW
[ 0.01714617  0.20603657  0.4504122  -0.48793542 -0.18296939  0.22635117
  0.21402073  0.30195048 -0.43837684 -0.15800509  0.06160438  0.44512972
  0.5648724  -0.5935517  -0.45970652  0.25720185  0.42276376  0.68241495
 -0.6886472  -

 [ 0.16944198  0.44285655  0.3209441  -0.63243604 -0.3790257 ]]
NEW
[ 0.14682646  0.5475349   0.64280677 -0.7150407  -0.49777597  0.08761055
  0.4143957   0.60915536 -0.8339049  -0.15433455  0.16944198  0.44285655
  0.3209441  -0.63243604 -0.3790257 ]

OLD
[[ 0.08761055  0.4143957   0.60915536 -0.8339049  -0.15433455]
 [ 0.16944198  0.44285655  0.3209441  -0.63243604 -0.3790257 ]
 [ 0.09107731  0.46558252  0.49899    -0.8338196  -0.29055646]]
NEW
[ 0.08761055  0.4143957   0.60915536 -0.8339049  -0.15433455  0.16944198
  0.44285655  0.3209441  -0.63243604 -0.3790257   0.09107731  0.46558252
  0.49899    -0.8338196  -0.29055646]

OLD
[[ 0.16944198  0.44285655  0.3209441  -0.63243604 -0.3790257 ]
 [ 0.09107731  0.46558252  0.49899    -0.8338196  -0.29055646]
 [ 0.31762922  0.4053853   0.5939043  -0.7664881  -0.16820337]]
NEW
[ 0.16944198  0.44285655  0.3209441  -0.63243604 -0.3790257   0.09107731
  0.46558252  0.49899    -0.8338196  -0.29055646  0.31762922  0.4053853
  0.5939043  -0.76648

  0.46558252  0.49899    -0.8338196  -0.29055646]

OLD
[[ 0.09107731  0.46558252  0.49899    -0.8338196  -0.29055646]
 [ 0.31762922  0.4053853   0.5939043  -0.7664881  -0.16820337]]
NEW
[ 0.09107731  0.46558252  0.49899    -0.8338196  -0.29055646  0.31762922
  0.4053853   0.5939043  -0.7664881  -0.16820337]

OLD
[[ 0.31762922  0.4053853   0.5939043  -0.7664881  -0.16820337]
 [ 0.0495452   0.4156018   0.3027171  -0.53303087 -0.35914588]]
NEW
[ 0.31762922  0.4053853   0.5939043  -0.7664881  -0.16820337  0.0495452
  0.4156018   0.3027171  -0.53303087 -0.35914588]

OLD
[[ 0.0495452   0.4156018   0.3027171  -0.53303087 -0.35914588]
 [ 0.0828135   0.25674146  0.57784665 -0.45841748 -0.35575786]]
NEW
[ 0.0495452   0.4156018   0.3027171  -0.53303087 -0.35914588  0.0828135
  0.25674146  0.57784665 -0.45841748 -0.35575786]

OLD
[[ 0.0828135   0.25674146  0.57784665 -0.45841748 -0.35575786]
 [ 0.21336383  0.17675139  0.66554105 -0.44346333 -0.34795162]]
NEW
[ 0.0828135   0.25674146  0.57784665 -0

 [ 0.09107731  0.46558252  0.49899    -0.8338196  -0.29055646]]
NEW
[ 0.14682646  0.5475349   0.64280677 -0.7150407  -0.49777597  0.08761055
  0.4143957   0.60915536 -0.8339049  -0.15433455  0.16944198  0.44285655
  0.3209441  -0.63243604 -0.3790257   0.09107731  0.46558252  0.49899
 -0.8338196  -0.29055646]

OLD
[[ 0.08761055  0.4143957   0.60915536 -0.8339049  -0.15433455]
 [ 0.16944198  0.44285655  0.3209441  -0.63243604 -0.3790257 ]
 [ 0.09107731  0.46558252  0.49899    -0.8338196  -0.29055646]
 [ 0.31762922  0.4053853   0.5939043  -0.7664881  -0.16820337]]
NEW
[ 0.08761055  0.4143957   0.60915536 -0.8339049  -0.15433455  0.16944198
  0.44285655  0.3209441  -0.63243604 -0.3790257   0.09107731  0.46558252
  0.49899    -0.8338196  -0.29055646  0.31762922  0.4053853   0.5939043
 -0.7664881  -0.16820337]

OLD
[[ 0.16944198  0.44285655  0.3209441  -0.63243604 -0.3790257 ]
 [ 0.09107731  0.46558252  0.49899    -0.8338196  -0.29055646]
 [ 0.31762922  0.4053853   0.5939043  -0.7664881  -0.